In [1]:
from hgmca import wavelets, gmca
import healpy as hp
import numpy as np
import time
%matplotlib inline

def NOTIMPLEMENTED():
    raise NotImplementedError('Must specify config path')
    
# Pull up a nice Planck colormap
from matplotlib.colors import ListedColormap
colombi1_cmap = ListedColormap(np.loadtxt("Planck_Parchment_RGB.txt")/255.)
colombi1_cmap.set_bad("gray") # color of missing pixels
colombi1_cmap.set_under("white") # color of background, necessary if you want to use

# Running GMCA to get CMB Reconstruction

__Authors:__ Sebastian Wagner-Carena and Max Hopkins

__Created:__ 2/15/2020

__Last Run:__ 2/16/2020

__Goals:__ Show how to go from measurements of the CMB to a reconstructed CMB map using GMCA.

__Requirements:__ Follow the installation instructions to install s2let and hgmca.

The first step to running GMCA on our maps is to transform our healpix maps into wavelet coefficients. With the AxisymWaveletTransformation in wavelets this is fairly easy.

For information on the parameters of the wavelet transformations being used here see the [s2let website](http://astro-informatics.github.io/s2let/).

In [2]:
# Specify the parameters of our wavelet transformation.
wav_b = 3
min_scale = 1
band_lim = 625

# Specify the path to the s2let installation
s2let_path = '/Users/maxhopkins/desktop/s2let'#NOTIMPLEMENTED()

# Samp of 0 uses the minimum sampling for each wavelet coefficient map that still preserves the fidelity of the 
# transformation. This saves memory
samp = 0
wav_class = wavelets.AxisymWaveletTransformation(wav_b,min_scale,band_lim,s2let_path=s2let_path,samp=samp)

Now we can run through each of fits maps (one per frequency) and turn them into wavelet coefficients

In [3]:
# We will call the input data in the wavelet space X.
X = []

# Specify a prefix were our wavelet coefficient maps will be saved
wav_map_prefix = 'temp/wav'

# We specify the list of maps
input_map_dir = '../maps/'
input_maps = [input_map_dir+'CMB_%d_GHZ.fits'%(freq) for freq in [23,90,120,160,210,280,350]]

# Get the wavelet coefficients for each map
for input_map in input_maps:
    print('Working on map %s... '%(input_map),end="")
    X.append(wav_class.get_wavelet_coeff(input_map,wav_map_prefix))
    print('Done')
    
# Turn our final X into a numpy array
X = np.array(X)

Working on map ../maps/CMB_23_GHZ.fits... 

FileNotFoundError: [Errno 2] No such file or directory: '/Users/maxhopkins/desktop/s2let/bin/s2let_transform_analysis_hpx_multi': '/Users/maxhopkins/desktop/s2let/bin/s2let_transform_analysis_hpx_multi'

Let's check that, if we take one of our maps at random, we get back an accurate reconstruction

In [ ]:
# Pick a frequency
check_index = 2
orig_map_path = input_maps[check_index]
print('Checking map %s'%(orig_map_path))

# Create a map from the wavelet coefficients. We have to specify the nside since it's not deterministic!
temp_map_path = 'temp/temp.fits'
nside = 256
wav_class.get_map_from_wavelet_coeff(temp_map_path,nside,wav_map_prefix,X[check_index])

# Now let's view our original map and the reconstructed map
original_map = hp.read_map(orig_map_path,verbose=False)
hp.mollview(original_map,min=-300,max=300,title='Original Map',cmap=colombi1_cmap,unit='$\mu K$')

# And let's look at the reconstructed map
reconstructed_map = hp.read_map(temp_map_path,verbose=False)
hp.mollview(reconstructed_map,min=-300,max=300,title='Reconstructed Map',cmap=colombi1_cmap,unit='$\mu K$')

# And finally at their difference
hp.mollview(original_map-reconstructed_map,min=-3,max=3,title='Difference Map',cmap=colombi1_cmap,unit='$\mu K$')


Now we can do a few iterations of GMCA to get the reconstructed map!

In [ ]:
# We have to specify a few parameters for GMCA. First, the number of source and the number of iterations.
n_sources = 5
n_iterations = 500

# Next the lambda parameters. We need a lam_s for the sparsity prior. We'll set this to 50 since that's the right
# order of magnitude.
lam_s = 80

# Next we need to specify the prior on our mixing matrix. We'll specify a VERY strong prior that we expect to see
# a source like the CMB, and then no priors beyond that.
A_p = np.zeros((X.shape[0],n_sources))
# Set the first column to the CMB. Since our maps are in K_cmb, this is just 1 for all columns (normalized such'
# that the L2 norm of the column is 1).
A_p[:,0] = 1/np.sqrt(X.shape[0])
# Make this prior very strong. We need one prior per column.
lam_p = [1e11,0.0,0.0,0.0,0.0]

# Finally we can specify a min_rmse_rate. This forces the algorithm to return the min rmse every certain number of
# steps. A rate of 100 usually works well.
min_rmse_rate = 100

# If you want to get the same results out every time you run the notebook, set the random seed to a number 
# that isn't 0
seed = 0

# Now we can run the algorithm and see what we get out! This will take a few minutes depending on the speed
# of your computer.
print('Running gmca'); start = time.time();
A,S = gmca.gmca(X,n_sources,n_iterations,A_p=A_p,lam_p=lam_p,lam_s=lam_s,min_rmse_rate=min_rmse_rate,seed=seed)
print('GMCA took %d seconds'%(time.time()-start))

Normally we would have to find the column that most correspond to the CMB, but given our very strong prior we know that it's the first column. So we can just pull out the corresponding source and look at the reconstructed map!

In [ ]:
# Create a map from the wavelet coefficients. We have to specify the nside since it's not deterministic!
cmb_path = 'temp/recon_cmb.fits'
nside = 256
# Note that, in K_CMB units, we get the CMB is the same at all frequencies. So we will get the same output
# no matter what frequency we chose here.
freq_index = 0
cmb_index = 0
wav_class.get_map_from_wavelet_coeff(cmb_path,nside,wav_map_prefix,A[freq_index,cmb_index]*S[cmb_index])

# Now let's view our cmb map.
cmb_map = hp.read_map(cmb_path,verbose=False)
hp.mollview(cmb_map,min=-300,max=300,title='CMB Map',cmap=colombi1_cmap,unit='$\mu K$')

To get good results will require running the algorithm for a bit longer than 400 steps. Thankfully, we have the converged mixing matrix on hand (run for 80000 steps).

In [ ]:
# Load the mixing matrix and calculate S as the pseudo inverse
A = np.load('../results/gmca/A_mat_gmca_lams_80_lamp_1e11.npy')
S = np.matmul(np.linalg.pinv(A),X)

# Create a map from the wavelet coefficients. We have to specify the nside since it's not deterministic!
cmb_path = 'temp/recon_cmb.fits'
nside = 256
# Note that, in K_CMB units, we get the CMB is the same at all frequencies. So we will get the same output
# no matter what frequency we chose here.
freq_index = 0
cmb_index = 0
wav_class.get_map_from_wavelet_coeff(cmb_path,nside,wav_map_prefix,A[freq_index,cmb_index]*S[cmb_index])

# Now let's view our cmb map.
cmb_map = hp.read_map(cmb_path,verbose=False)
hp.mollview(cmb_map,min=-300,max=300,title='CMB Map',cmap=colombi1_cmap,unit='$\mu K$')